In [1]:
#from gaia_tools import query
#import gaia_tools.load as gload
from astroquery.gaia import Gaia
import matplotlib
#matplotlib.use("AGG")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table
import pyexcel
from astropy import units as u
from astropy.coordinates import SkyCoord

import sklearn
from sklearn import cluster
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.patches import Circle
#from matplotlib import patches

%pylab inline

#import pywt
from galpy.util import bovy_plot

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Populating the interactive namespace from numpy and matplotlib


In [2]:

# csv-pd set up for DBSCANN
GlobClust_Log_ra_dec = pd.read_csv('GlobClust_ra-dec.csv')
GlobClust_Log_r_t = GlobClust_Log_ra_dec.loc[0:0,"r_t"]
GlobClust_Log_radec = GlobClust_Log_ra_dec.loc[0:0,"RA":"DEC"]
GlobClust_Log_ID = GlobClust_Log_ra_dec.loc[0:0,"ID"]
GlobClust_Log_r_c = GlobClust_Log_ra_dec.loc[0:0,"r_c"]
GlobClust_Log_r_t = GlobClust_Log_ra_dec.loc[0:0,"r_t"]

coord_ra_dec_deg=[]
coord_ra_dec = SkyCoord(GlobClust_Log_radec.loc[:,"RA"], GlobClust_Log_radec.loc[:,"DEC"], frame='icrs')

GlobClust_Log_dec = coord_ra_dec.dec.deg
GlobClust_Log_ra = coord_ra_dec.ra.deg

In [12]:
#CONE SEARCH
job = Gaia.launch_job_async("SELECT top 100 gaia.ra, gaia.dec, gaia.l, gaia.b, gaia.pmra, \
gaia.pmdec, gaia.phot_g_mean_mag, gaia.phot_rp_mean_mag, gaia.phot_bp_mean_mag, \
gaia.parallax,gaia.parallax_error,gaia.a_g_val ,panstarrs1.g_mean_psf_mag, \
panstarrs1.g_mean_psf_mag_error,panstarrs1.r_mean_psf_mag,panstarrs1.r_mean_psf_mag_error, \
panstarrs1.i_mean_psf_mag,panstarrs1.i_mean_psf_mag_error FROM gaiadr2.gaia_source \
AS gaia INNER JOIN gaiadr2.panstarrs1_best_neighbour AS panstarrs1_match ON \
panstarrs1_match.source_id = gaia.source_id INNER JOIN gaiadr2.panstarrs1_original_valid \
AS panstarrs1 ON panstarrs1.obj_id = panstarrs1_match.original_ext_source_id \
WHERE 1=CONTAINS(POINT('ICRS',gaia.ra,gaia.dec),CIRCLE('ICRS',%f,%f,%f))" % \
(GlobClust_Log_ra,GlobClust_Log_dec,GlobClust_Log_r_t/60*15),dump_to_file=False)
data = job.get_results()

Launched query: 'SELECT top 100 gaia.ra, gaia.dec, gaia.l, gaia.b, gaia.pmra, gaia.pmdec, gaia.phot_g_mean_mag, gaia.phot_rp_mean_mag, gaia.phot_bp_mean_mag, gaia.parallax,gaia.parallax_error,gaia.a_g_val ,panstarrs1.g_mean_psf_mag, panstarrs1.g_mean_psf_mag_error,panstarrs1.r_mean_psf_mag,panstarrs1.r_mean_psf_mag_error, panstarrs1.i_mean_psf_mag,panstarrs1.i_mean_psf_mag_error FROM gaiadr2.gaia_source AS gaia INNER JOIN gaiadr2.panstarrs1_best_neighbour AS panstarrs1_match ON panstarrs1_match.source_id = gaia.source_id INNER JOIN gaiadr2.panstarrs1_original_valid AS panstarrs1 ON panstarrs1.obj_id = panstarrs1_match.original_ext_source_id WHERE 1=CONTAINS(POINT('ICRS',gaia.ra,gaia.dec),CIRCLE('ICRS',13.188500,-26.582611,3.298176))'
Retrieving async. results...
/usr/local/lib/python3.6/site-packages/astropy/io/votable/tree.py:1640: W27: None:3:0: W27: COOSYS deprecated in VOTable 1.2
  warn_or_raise(W27, W27, (), config, pos)

/usr/local/lib/python3.6/site-packages/astropy/io/votable/

In [13]:
print(data[0]['ra'])

16.207946523380006


In [15]:
#BOX SEARCH (with parameters as string)

params=('gaia.ra, gaia.dec, gaia.l, gaia.b, \
gaia.pmra, gaia.pmdec, gaia.phot_g_mean_mag, gaia.phot_rp_mean_mag, gaia.phot_bp_mean_mag, \
gaia.parallax,gaia.parallax_error,gaia.a_g_val ,panstarrs1.g_mean_psf_mag, \
panstarrs1.g_mean_psf_mag_error,panstarrs1.r_mean_psf_mag,panstarrs1.r_mean_psf_mag_error, \
panstarrs1.i_mean_psf_mag,panstarrs1.i_mean_psf_mag_error')

job = Gaia.launch_job_async("SELECT top 100 %s FROM gaiadr2.gaia_source AS gaia \
INNER JOIN gaiadr2.panstarrs1_best_neighbour AS panstarrs1_match ON \
panstarrs1_match.source_id = gaia.source_id INNER JOIN gaiadr2.panstarrs1_original_valid \
AS panstarrs1 ON panstarrs1.obj_id = panstarrs1_match.original_ext_source_id WHERE gaia.ra \
BETWEEN %f AND %f AND gaia.dec BETWEEN %f AND %f" % \
(params,GlobClust_Log_ra-GlobClust_Log_r_t/60*15,GlobClust_Log_ra+GlobClust_Log_r_t/60*15, \
 GlobClust_Log_dec-GlobClust_Log_r_t/60*15,GlobClust_Log_dec+GlobClust_Log_r_t/60*15),dump_to_file=False)
data = job.get_results()

Launched query: 'SELECT top 100 gaia.ra, gaia.dec, gaia.l, gaia.b, gaia.pmra, gaia.pmdec, gaia.phot_g_mean_mag, gaia.phot_rp_mean_mag, gaia.phot_bp_mean_mag, gaia.parallax,gaia.parallax_error,gaia.a_g_val ,panstarrs1.g_mean_psf_mag, panstarrs1.g_mean_psf_mag_error,panstarrs1.r_mean_psf_mag,panstarrs1.r_mean_psf_mag_error, panstarrs1.i_mean_psf_mag,panstarrs1.i_mean_psf_mag_error FROM gaiadr2.gaia_source AS gaia INNER JOIN gaiadr2.panstarrs1_best_neighbour AS panstarrs1_match ON panstarrs1_match.source_id = gaia.source_id INNER JOIN gaiadr2.panstarrs1_original_valid AS panstarrs1 ON panstarrs1.obj_id = panstarrs1_match.original_ext_source_id WHERE gaia.ra BETWEEN 9.890324 AND 16.486676 AND gaia.dec BETWEEN -29.880787 AND -23.284435'
Retrieving async. results...
/usr/local/lib/python3.6/site-packages/astropy/io/votable/tree.py:1640: W27: None:3:0: W27: COOSYS deprecated in VOTable 1.2
  warn_or_raise(W27, W27, (), config, pos)

/usr/local/lib/python3.6/site-packages/astropy/io/votable/tr

In [11]:
print(data[0]['ra'])

9.890583995614403
